<a href="https://colab.research.google.com/github/RaqeebShaikh12/SpottablAssignment/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd

with open('domainTags2.json', encoding='utf-8') as f:
    data = json.load(f)
    
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Function to find similar companies based on tags and description
def find_similar_companies(company_name):
    # Find the company object with the given name
    company = None
    for c in df:
        if company_name.lower() in c['companyName']:
            company = c
            break

    if not company:
        return []

    # Find companies with similar tags
    similar_companies = []
    for c in df:
        if c == company:
            continue
        if any(tag in c['tags'] for tag in company['tags']):
            similarity = SequenceMatcher(None, company['companyDesc'], c['companyDesc']).ratio()
            if similarity > 0.5: # adjust threshold as needed
                similar_companies.append(c)

    return similar_companies

# Example usage
similar_companies = find_similar_companies('Swiggy')
print([c['companyName'] for c in similar_companies])

In [ ]:
from datetime import datetime

# Function to calculate growth rate of a company
def calculate_growth_rate(company):
    if not company['fundingList']:
        return None
    latest_funding = max(company['fundingList'], key=lambda x: x['fundedYear'])
    founded_year = datetime.strptime(company['foundedDate'], '%Y-%m-%d').year
    years_since_founded = datetime.now().year - founded_year
    growth_rate = latest_funding['raisedAmount'] / years_since_founded
    return growth_rate

# Function to find companies with fastest and slowest growth based on a domain
def find_growth_companies(domain):
    # Find companies with the given domain
    domain_companies = [c for c in df if domain in c['tags']]

    # Calculate growth rates of all companies
    companies_growth_rates = [(c, calculate_growth_rate(c)) for c in domain_companies]

    # Sort by growth rate
    companies_growth_rates = sorted(companies_growth_rates, key=lambda x: x[1])

    return companies_growth_rates[0][0], companies_growth_rates[-1][0]

# Example usage
fastest_growing, slowest_growing = find_growth_companies('e-commerce')
print(f'Fastest growing company: {fastest_growing["companyName"]}')
print(f'Slowest growing company: {slowest_growing["companyName"]}')